# PySpark Tutorial - Applications
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

In [1]:
from pyspark import SparkContext, SparkConf
import numpy as np
import operator

In [2]:
conf=SparkConf().setAppName("pyspark tutorial").setMaster("local[*]")
sc = SparkContext(conf=conf)

## WordCount

In [3]:
lines=sc.textFile("hamlet.txt")

In [4]:
lines.take(3)

['THE TRAGEDY OF HAMLET, PRINCE OF DENMARK', '', '']

In [5]:
counts = lines.flatMap(lambda line: line.split())\
              .map(lambda word: (word,1))\
              .reduceByKey(operator.add)

In [6]:
counts.take(5)

[('TRAGEDY', 1), ('OF', 2), ('PRINCE', 1), ('Shakespeare', 1), ('Dramatis', 1)]

In [7]:
lines.flatMap(lambda line: line.split())\
              .map(lambda word: word.lower())\
              .map(lambda word: (word,1))\
              .reduceByKey(operator.add)\
              .sortBy(lambda x: x[1], ascending=False)\
              .take(5)

[('the', 1083), ('and', 939), ('to', 727), ('of', 670), ('a', 540)]

## Page Rank

We represent our graph as a simple vertex-edge-list with the edges stored as tuples. Because each node is a Key-Value, we can directly parallelize the graph and then operate on it using K-V operation.s

In [8]:
graph = sc.parallelize([
    ('A', ('B')),
    ('B', ('A', 'C')),
    ('C', ('A', 'D')),
    ('D', ('A'))
])

The current page rank is represented as pairs of the node name the current value. We initialize the page rank to 1.0

In [9]:
ranks = graph.map( lambda node: (node[0], 1.0))

In [10]:
ranks.collect()

[('A', 1.0), ('B', 1.0), ('C', 1.0), ('D', 1.0)]

The current page will contribute its current rank divided by the number of out edges to each node. Because the edge list indicates the destination node, this will produce pairs of values indicating the node and the contribution

In [11]:
def computeContrib(edges, rank):
    return ( (e, rank/len(edges)) for e in edges )

In [12]:
list(computeContrib(('A','D'), 1.0))

[('A', 0.5), ('D', 0.5)]

We need to use both the graph and the current rank information -- we do this using a `join`

In [13]:
graph.join(ranks).collect()

[('B', (('A', 'C'), 1.0)),
 ('D', ('A', 1.0)),
 ('C', (('A', 'D'), 1.0)),
 ('A', ('B', 1.0))]

Now, to compute the contribution of each link for each node, we use use `computeContrib` for that nodes information (edge list & rank). Here's an example of that happening in a single step:

In [14]:
graph.join(ranks).flatMap(lambda node: computeContrib(node[1][0],node[1][1])).collect()

[('A', 0.5), ('C', 0.5), ('A', 1.0), ('A', 0.5), ('D', 0.5), ('B', 1.0)]

Now, we reduce the values by the key and sum up the contributions. For example,

In [15]:
graph.join(ranks).flatMap(lambda node: computeContrib(node[1][0],node[1][1]))\
    .reduceByKey(operator.add).collect()

[('B', 1.0), ('D', 0.5), ('C', 0.5), ('A', 2.0)]

These contributions are used to calculate the final page rank.

We can then perform the rank update operation multiple times until we converge to an answer. In our case, we're going to just run the code 5 times.

In [16]:
for itr in range(5):
    print("=== Iteration {} ====".format(itr))
    contribs = graph.join(ranks).\
       flatMap(lambda node: computeContrib(node[1][0], node[1][1]))
    print("Contribs are", contribs.collect())
    ranks = contribs.reduceByKey(operator.add).\
                     mapValues(lambda rank: rank * 0.85 + 0.15)
    print("Ranks are", ranks.collect())
print("====")
print("Final rank:", ranks.collect())

=== Iteration 0 ====
Contribs are [('A', 0.5), ('C', 0.5), ('A', 1.0), ('A', 0.5), ('D', 0.5), ('B', 1.0)]
Ranks are [('B', 1.0), ('D', 0.575), ('C', 0.575), ('A', 1.8499999999999999)]
=== Iteration 1 ====
Contribs are [('A', 0.5), ('C', 0.5), ('B', 1.8499999999999999), ('A', 0.575), ('A', 0.2875), ('D', 0.2875)]
Ranks are [('B', 1.7224999999999997), ('A', 1.3081249999999998), ('D', 0.394375), ('C', 0.575)]
=== Iteration 2 ====
Contribs are [('A', 0.8612499999999998), ('C', 0.8612499999999998), ('A', 0.394375), ('A', 0.2875), ('D', 0.2875), ('B', 1.3081249999999998)]
Ranks are [('B', 1.2619062499999998), ('D', 0.394375), ('C', 0.8820624999999999), ('A', 1.4616562499999997)]
=== Iteration 3 ====
Contribs are [('A', 0.394375), ('A', 0.6309531249999999), ('C', 0.6309531249999999), ('A', 0.44103124999999993), ('D', 0.44103124999999993), ('B', 1.4616562499999997)]
Ranks are [('D', 0.5248765624999999), ('B', 1.3924078124999997), ('C', 0.6863101562499999), ('A', 1.3964054687499998)]
=== Itera